In [ ]:
import sqlite3

import tkinter as tk

from tkinter import messagebox

import requests


# Create a SQLite database connection

conn = sqlite3.connect('foreign_trading_system.db')

c = conn.cursor()


# Create table for storing user credentials

c.execute('''CREATE TABLE IF NOT EXISTS users

             (id INTEGER PRIMARY KEY AUTOINCREMENT, username TEXT, password TEXT)''')


# Create table for storing currency details

c.execute('''CREATE TABLE IF NOT EXISTS currencies

             (id INTEGER PRIMARY KEY AUTOINCREMENT, code TEXT, name TEXT)''')


# Create table for storing exchange rates

c.execute('''CREATE TABLE IF NOT EXISTS exchange_rates

             (id INTEGER PRIMARY KEY AUTOINCREMENT, from_currency_id INTEGER, to_currency_id INTEGER,

              rate REAL)''')


# Create table for storing transactions

c.execute('''CREATE TABLE IF NOT EXISTS transactions

             (id INTEGER PRIMARY KEY AUTOINCREMENT, user_id INTEGER, from_currency_id INTEGER,

              to_currency_id INTEGER, amount REAL, converted_amount REAL)''')


class ForeignTradingSystem(tk.Tk):

    def __init__(self):

        super().__init__()

        self.title("Foreign Trading System")

        self.geometry("400x300")


        self.current_form = None

        self.switch_form(LoginForm)


    def switch_form(self, form_class):

        new_form = form_class(self)

        if self.current_form is not None:

            self.current_form.pack_forget()

        self.current_form = new_form

        self.current_form.pack()


class LoginForm(tk.Frame):

    def __init__(self, master):

        super().__init__(master)

        tk.Label(self, text="Username:").pack()

        self.username_entry = tk.Entry(self)

        self.username_entry.pack()

        tk.Label(self, text="Password:").pack()

        self.password_entry = tk.Entry(self, show="*")

        self.password_entry.pack()

        tk.Button(self, text="Login", command=self.login).pack()

        tk.Button(self, text="Signup", command=lambda: master.switch_form(SignupForm)).pack()


    def login(self):

        username = self.username_entry.get()

        password = self.password_entry.get()

        # Authenticate user against database

        c.execute("SELECT * FROM users WHERE username=? AND password=?", (username, password))

        user = c.fetchone()

        if user:

            self.master.switch_form(MainMenu)

        else:

            messagebox.showerror("Error", "Invalid Username and Password")


class SignupForm(tk.Frame):

    def __init__(self, master):

        super().__init__(master)

        tk.Label(self, text="Username:").pack()

        self.username_entry = tk.Entry(self)

        self.username_entry.pack()

        tk.Label(self, text="Password:").pack()

        self.password_entry = tk.Entry(self, show="*")

        self.password_entry.pack()

        tk.Button(self, text="Signup", command=self.signup).pack()

        tk.Button(self, text="Cancel", command=lambda: master.switch_form(LoginForm)).pack()


    def signup(self):

        username = self.username_entry.get()

        password = self.password_entry.get()

        if username and password:

            # Check if username already exists

            c.execute("SELECT * FROM users WHERE username=?", (username,))

            existing_user = c.fetchone()

            if existing_user:

                messagebox.showerror("Error", "Username already exists. Please choose a different username.")

            else:

                # Insert new user into the database

                c.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))

                conn.commit()

                messagebox.showinfo("Signup", "Account created successfully. You can now login.")

                self.master.switch_form(LoginForm)

        else:

            messagebox.showerror("Error", "Please enter a username and password.")


class MainMenu(tk.Frame):

    def __init__(self, master):

        super().__init__(master)

        tk.Label(self, text="Main Menu").pack()

        tk.Button(self, text="View Exchange Rates", command=lambda: master.switch_form(ExchangeRatesForm)).pack()

        tk.Button(self, text="Make Transaction", command=lambda: master.switch_form(TransactionForm)).pack()

        tk.Button(self, text="Logout", command=lambda: master.switch_form(LoginForm)).pack()


class ExchangeRatesForm(tk.Frame):

    def __init__(self, master):

        super().__init__(master)

        tk.Label(self, text="Exchange Rates").pack()

        exchange_rates = self.get_exchange_rates()

        for rate in exchange_rates:

            tk.Label(self, text=f"{rate['from_currency']} to {rate['to_currency']}: {rate['rate']}").pack()

        tk.Button(self, text="Back", command=lambda: master.switch_form(MainMenu)).pack()


    def get_exchange_rates(self):

        # Mock function to fetch exchange rates from an API (Replace with actual API call)

        # Example response: [{'from_currency': 'USD', 'to_currency': 'EUR', 'rate': 0.82}, ...]

        # Here, replace this mock function with actual API call to get exchange rates

        return [{'from_currency': 'USD', 'to_currency': 'EUR', 'rate': 0.82},

                {'from_currency': 'USD', 'to_currency': 'GBP', 'rate': 0.74},

                {'from_currency': 'EUR', 'to_currency': 'GBP', 'rate': 0.90}]


class TransactionForm(tk.Frame):

    def __init__(self, master):

        super().__init__(master)

        self.master = master

        self.currencies = self.get_currencies()

        self.from_currency = tk.StringVar(self)

        self.to_currency = tk.StringVar(self)

        self.amount_entry = tk.Entry(self)

        self.from_currency.set(self.currencies[0])  # Set default values

        self.to_currency.set(self.currencies[1])    # Set default values

        tk.Label(self, text="From Currency:").pack()

        tk.OptionMenu(self, self.from_currency, *self.currencies).pack()

        tk.Label(self, text="To Currency:").pack()

        tk.OptionMenu(self, self.to_currency, *self.currencies).pack()

        tk.Label(self, text="Amount:").pack()

        self.amount_entry.pack()

        tk.Button(self, text="Convert", command=self.convert).pack()

        tk.Button(self, text="Back", command=lambda: master.switch_form(MainMenu)).pack()


    def get_currencies(self):

        # Mock function to fetch currencies from the database (Replace with actual query)

        # Example: ['USD', 'EUR', 'GBP']

        # Here, replace this mock function with actual database query to get currency codes

        return ['USD', 'EUR', 'GBP']


    def convert(self):

        from_currency = self.from_currency.get()

        to_currency = self.to_currency.get()

        amount = float(self.amount_entry.get())

        # Perform currency conversion (Replace with actual logic)

        # Here, you'll need to fetch exchange rate from the database based on from_currency and to_currency

        # and then calculate the converted amount

        converted_amount = amount * 0.82  # Mock conversion, replace with actual calculation

        messagebox.showinfo("Conversion Result", f"{amount} {from_currency} = {converted_amount} {to_currency}")


if __name__ == "__main__":

    app = ForeignTradingSystem()

    app.mainloop()